In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:

stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 73345071,
 'calculationPrice': 'close',
 'change': -1.28,
 'changePercent': -0.00843,
 'close': 156.78,
 'closeSource': 'afioicfl',
 'closeTime': 1662871930405,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 156.306,
 'delayedPriceTime': 1697550772700,
 'extendedChange': -0.17,
 'extendedChangePercent': -0.00104,
 'extendedPrice': 159.48,
 'extendedPriceTime': 1737602089912,
 'high': 161.65,
 'highSource': 'aie  i1uer5mpynteec ldd',
 'highTime': 1699746853343,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 160.49,
 'iexCloseTime': 1697482112484,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 161.13,
 'iexOpenTime': 1665672339520,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1685221846235,
 'latestPrice': 157.01,
 'latestSource': 'Close',
 'latestTime': 'July 22, 2022',
 'latestUpdate': 1691594777857,
 'latestVolume': 69196015,
 'low': 

In [4]:
pe_ratio = data['peRatio']
pe_ratio

25.78

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,125.17,30.58,N/A
1,AAL,14.35,-4.9,N/A
2,AAP,198.75,22.26,N/A
3,AAPL,154.69,25.57,N/A
4,ABBV,154.83,21.77,N/A
...,...,...,...,...
496,YUM,121.92,21.69,N/A
497,ZBH,110.00,111,N/A
498,ZBRA,335.96,21.68,N/A
499,ZION,53.10,5.55,N/A


In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:20]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [9]:
portfolio_input()

Enter the value of your portfolio:10000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,110.10,1,4
1,NRG,37.33,2.24,13
2,EBAY,48.28,2.6,10
3,AMZN,126.43,3.02,3
4,COF,111.31,4.37,4
5,NUE,124.16,4.44,4
6,PVH,63.34,4.56,7
7,F,12.85,4.67,38
8,AIG,51.57,4.69,9
9,KSS,30.54,4.78,16


In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,191.300,N/A,59.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,284.490,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,289.090,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,96.900,N/A,49.28,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.630,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,51.260,N/A,14.77,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.900,N/A,32.06,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,34.050,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,104.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.716567
1      0.033932
2       0.52495
3       0.59481
4      0.530938
         ...   
496    0.528942
497    0.982036
498    0.522954
499    0.077844
500    0.862275
Name: PE Percentile, Length: 501, dtype: object
0      0.811377
1       0.05988
2      0.637725
3       0.97006
4      0.924152
         ...   
496    0.055888
497    0.280439
498    0.762475
499    0.179641
500    0.938124
Name: PB Percentile, Length: 501, dtype: object
0      0.822355
1      0.013972
2      0.165669
3      0.846307
4      0.764471
         ...   
496    0.786427
497    0.528942
498    0.533932
499    0.433134
500     0.94012
Name: PS Percentile, Length: 501, dtype: object
0      0.808383
1      0.996008
2       0.41517
3      0.764471
4      0.381238
         ...   
496    0.768463
497    0.678643
498    0.694611
499     0.05988
500    0.928144
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.804391
1      0.027944
2      0.149701
3      0.914172
4      0.672655
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.65,N/A,30.18,0.716567,7.30,0.811377,5.7900,0.822355,21.630189,0.808383,11.010277,0.804391,N/A
1,AAL,14.05,N/A,-4.95,0.033932,-1.08,0.05988,0.2207,0.013972,104.150332,0.996008,0.794109,0.027944,N/A
2,AAP,200.25,N/A,22.07,0.52495,4.16,0.637725,1.0800,0.165669,12.159862,0.41517,2.577580,0.149701,N/A
3,AAPL,157.41,N/A,25.41,0.59481,38.00,0.97006,6.5500,0.846307,19.816891,0.764471,15.437293,0.914172,N/A
4,ABBV,153.30,N/A,22.16,0.530938,16.12,0.924152,4.8000,0.764471,11.509501,0.381238,8.516403,0.672655,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.33,N/A,22.14,0.528942,-4.11,0.055888,5.1700,0.786427,19.939436,0.768463,9.540429,0.740519,N/A
497,ZBH,111.00,N/A,111.13,0.982036,1.90,0.280439,2.9700,0.528942,17.190444,0.678643,5.129922,0.367265,N/A
498,ZBRA,337.35,N/A,22.05,0.522954,6.20,0.762475,3.0000,0.533932,17.498352,0.694611,7.149749,0.54491,N/A
499,ZION,52.98,N/A,5.57,0.077844,1.40,0.179641,2.3800,0.433134,4.203619,0.05988,2.131789,0.129741,N/A


In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.65,N/A,30.18,0.716567,7.30,0.811377,5.7900,0.822355,21.630189,0.808383,11.010277,0.804391,0.792615
1,AAL,14.05,N/A,-4.95,0.033932,-1.08,0.05988,0.2207,0.013972,104.150332,0.996008,0.794109,0.027944,0.226347
2,AAP,200.25,N/A,22.07,0.52495,4.16,0.637725,1.0800,0.165669,12.159862,0.41517,2.577580,0.149701,0.378643
3,AAPL,157.41,N/A,25.41,0.59481,38.00,0.97006,6.5500,0.846307,19.816891,0.764471,15.437293,0.914172,0.817964
4,ABBV,153.30,N/A,22.16,0.530938,16.12,0.924152,4.8000,0.764471,11.509501,0.381238,8.516403,0.672655,0.654691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.33,N/A,22.14,0.528942,-4.11,0.055888,5.1700,0.786427,19.939436,0.768463,9.540429,0.740519,0.576048
497,ZBH,111.00,N/A,111.13,0.982036,1.90,0.280439,2.9700,0.528942,17.190444,0.678643,5.129922,0.367265,0.567465
498,ZBRA,337.35,N/A,22.05,0.522954,6.20,0.762475,3.0000,0.533932,17.498352,0.694611,7.149749,0.54491,0.611776
499,ZION,52.98,N/A,5.57,0.077844,1.40,0.179641,2.3800,0.433134,4.203619,0.05988,2.131789,0.129741,0.176048


In [22]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:20]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop('index', axis=1, inplace = True)

<ipython-input-22-ff156822ad67>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values(by = 'RV Score', inplace = True)


In [23]:
portfolio_input()

Enter the value of your portfolio:10000


In [24]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GOOGL,111.40,4,1.03,0.045908,0.2970,0.063872,0.2815,0.01996,-0.492594,0.015968,-0.300007,0.001996,0.029541
1,KSS,30.79,16,4.66,0.063872,0.8920,0.095808,0.2032,0.00998,3.330407,0.035928,0.984659,0.041916,0.049501
2,AIG,53.80,9,4.60,0.061876,0.7674,0.07984,0.7819,0.10978,2.691816,0.025948,0.737315,0.021956,0.05988
3,TRV,156.53,3,11.14,0.221557,0.1704,0.061876,0.1080,0.003992,0.726824,0.017964,0.105051,0.003992,0.061876
4,UNM,32.77,15,7.30,0.117764,0.6301,0.075848,0.5493,0.06986,3.756862,0.047904,0.544352,0.00998,0.064271
5,GM,35.73,13,5.86,0.08982,0.8424,0.08982,0.4016,0.037924,1.921847,0.01996,1.617839,0.091816,0.065868
6,GPS,9.86,50,-51.48,0.007984,1.4300,0.187625,0.2151,0.011976,5.083764,0.0998,0.741951,0.023952,0.066267
7,PVH,64.73,7,4.50,0.055888,0.8131,0.087824,0.4588,0.051896,4.646788,0.07984,1.075201,0.055888,0.066267
8,LNC,51.53,9,7.22,0.115768,0.5821,0.067864,0.4510,0.047904,5.086568,0.101796,0.424029,0.005988,0.067864
9,PRU,98.27,5,7.77,0.131737,0.8446,0.091816,0.5460,0.067864,4.996090,0.095808,0.536299,0.007984,0.079042


In [25]:
writer = pd.ExcelWriter('value_strategy.xls')
#rv_dataframe.to_excel('value_strategy.xls', index = False)
rv_dataframe.to_excel(writer, sheet_name='Sheet1', index = False)

writer.close()


C:\Users\Home\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  if (await self.run_code(code, result,  async_=asy)):


In [309]:
#background_color = '#0a0a23'
#font_color = '#ffffff'
#
#string_template = writer.book.add_format(
#        {
#            'font_color': font_color,
#            'bg_color': background_color,
#            'border': 1
#        }
#    )
#
#dollar_template = writer.book.add_format(
#        {
#            'num_format':'$0.00',
#            'font_color': font_color,
#            'bg_color': background_color,
#            'border': 1
#        }
#    )
#
#integer_template = writer.book.add_format(
#        {
#            'num_format':'0',
#            'font_color': font_color,
#            'bg_color': background_color,
#            'border': 1
#        }
#    )
#
#float_template = writer.book.add_format(
#        {
#            'num_format':'0',
#            'font_color': font_color,
#            'bg_color': background_color,
#            'border': 1
#        }
#    )
#
#percent_template = writer.book.add_format(
#        {
#            'num_format':'0.0%',
#            'font_color': font_color,
#            'bg_color': background_color,
#            'border': 1
#        }
#    )

In [310]:
#column_formats = {
#                    'A': ['Ticker', string_template],
#                    'B': ['Price', dollar_template],
#                    'C': ['Number of Shares to Buy', integer_template],
#                    'D': ['Price-to-Earnings Ratio', float_template],
#                    'E': ['PE Percentile', percent_template],
#                    'F': ['Price-to-Book Ratio', float_template],
#                    'G': ['PB Percentile',percent_template],
#                    'H': ['Price-to-Sales Ratio', float_template],
#                    'I': ['PS Percentile', percent_template],
#                    'J': ['EV/EBITDA', float_template],
#                    'K': ['EV/EBITDA Percentile', percent_template],
#                    'L': ['EV/GP', float_template],
#                    'M': ['EV/GP Percentile', percent_template],
#                    'N': ['RV Score', percent_template]
#                 }
#
#for column in column_formats.keys():
#    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
#    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [311]:
#writer.save()